In [1]:
! git clone https://github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.84 MiB | 41.67 MiB/s, done.
Resolving deltas: 100% (570/570), done.


In [2]:
! pip install keras==2.1.3
! pip install tensorflow==1.9.0
! pip install -r './Mask_RCNN/requirements.txt' 

     |████████████████████████████████| 327kB 2.8MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.3 which is incompatible.
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 51.1MB 60kB/s 
     |████████████████████████████████| 573kB 36.7MB/s 
     |████████████████████████████████| 3.3MB 44.2MB/s 
ERROR: google-auth 1.7.2 has requirement setuptools>=40.3.0, but you'll have setuptools 39.1.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: setuptools 46.0.0
    Uninstalling setuptools-46.0.0:
      Successfully uninstalled setuptools-46.0.0
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: tensorflow 2.2.0rc1
    Uninsta

     |████████████████████████████████| 204kB 2.8MB/s 
     |████████████████████████████████| 163kB 46.5MB/s 


In [3]:
%cd Mask_RCNN/ 
! python3 setup.py install

/content/Mask_RCNN
running install
running bdist_egg
running egg_info
creating mask_rcnn.egg-info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/mrcnn
copying mrcnn/visualize.py -> build/lib/mrcnn
copying mrcnn/config.py -> build/lib/mrcnn
copying mrcnn/parallel_model.py -> build/lib/mrcnn
copying mrcnn/__init__.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
copying mrcnn/utils.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist

In [4]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [5]:
# -*- coding: utf-8 -*-
"""
Created on Sat Mar  7 22:38:21 2020

@author: jiayi
"""

import os
import sys
import cv2
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
import colorsys
import argparse
#import imutils
import random
import cv2
import os
import time
from matplotlib import pyplot
from matplotlib.patches import Rectangle
from keras.models import load_model
from os import listdir
import random


ROOT_DIR = os.path.abspath(".")

sys.path.append(ROOT_DIR)
sys.path.insert(0, '/content/Mask_RCNN')
from mrcnn.config import Config
from mrcnn import visualize
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.model import mold_image
from mrcnn.utils import compute_ap


import mrcnn
from mrcnn.utils import Dataset
from mrcnn.model import MaskRCNN
from mrcnn import model as modellib, utils

Using TensorFlow backend.


In [6]:
! git clone https://github.com/JWang5/aridDataset.git

Cloning into 'aridDataset'...
remote: Enumerating objects: 16903, done.
remote: Total 16903 (delta 0), reused 0 (delta 0), pack-reused 16903
Receiving objects: 100% (16903/16903), 632.22 MiB | 45.80 MiB/s, done.
Resolving deltas: 100% (737/737), done.
Checking out files: 100% (15745/15745), done.


In [7]:
DATA_ROOT_PATH = "/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset"
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
utils.download_trained_weights("/content/Mask_RCNN/mask_rcnn_coco.h5")

... done downloading pretrained model!


In [8]:
def get_object_classes():  
    classes = []
    for root,_,files in os.walk(DATA_ROOT_PATH):
      if root.split('/')[-1] == 'crops':
        for o in os.listdir(root):
          if o not in classes:
            classes.append(o)
    return classes
    
    
def get_paths():
    image_paths =[]
    json_paths = []
    paths = list()
    #path = os.path.join(DATA_ROOT_PATH, 'Exp_1')
    for root,_,files in os.walk(DATA_ROOT_PATH):
      random.shuffle(files)
      for f in files:
        if f.split('.')[-1] == 'png' and root.split('/')[-1] == 'rgb':
          f = os.path.join(root, f)
          #image_paths.append(f)
          image_id = f.split('/')[-1].split('.')[0]
          anno_path = os.path.join(root.split('rgb')[0], image_id) + '.json'
          path = {"image_path": f, "anno_path": anno_path}
          paths.append(path)
    random.shuffle(paths)
    print(paths)
    return paths

def get_jsons():
    jsons = []
    for root,_,files in os.walk(DATA_ROOT_PATH):
        for f in files:
            if f.split('.')[-1] == 'json':
                f = os.path.join(root, f)
                jsons.append(f)
    return jsons
    
def find_image(image_list, image_id):
    for img in image_list:
        if img['id'] == image_id:
            return img

paths = get_paths()
print(len(paths))

[{'image_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_3/wp_3_1_5/rgb/wp_3_1_5_011.png', 'anno_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_3/wp_3_1_5/wp_3_1_5_011.json'}, {'image_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_1/wp_1_4_29/rgb/wp_1_4_29_005.png', 'anno_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_1/wp_1_4_29/wp_1_4_29_005.json'}, {'image_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_1/wp_1_3_28/rgb/wp_1_3_28_010.png', 'anno_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_1/wp_1_3_28/wp_1_3_28_010.json'}, {'image_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_3/wp_3_2_52/rgb/wp_3_2_52_012.png', 'anno_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_3/wp_3_2_52/wp_3_2_52_012.json'}, {'image_path': '/content/Mask_RCNN/aridDataset/arid_40k_scene_dataset/Exp_1/wp_1_1_42/rgb/wp_1_1_42_015.png', 'anno_path': '/content/Mask_R

In [9]:


class AridDataSet(utils.Dataset):
    def load_dataset(self, is_train=True):
        #index = 1
        #add all classes 
        classes = get_object_classes()
        for c in classes:   
            self.add_class("arid", classes.index(c), c)
            #index += 1
        #image_paths, json_paths = get_paths()
        #count = 0
        for i in range(0,len(paths)):
            #count += 1
            #index = path.index(path)
            if is_train and i >= len(paths)*0.8:
                continue
            if not is_train and i < len(paths)*0.8:
                continue
            #image_id = image.split('/')[-1].split('.')[0]
            ann_path = paths[i]["anno_path"]
            self.add_image('arid', image_id=i, path=paths[i]["image_path"], annotation=ann_path)
    
    def extract_boxes(self, filename):
        boxes=list()
        classes = list()
        annotations = json.load(open(filename))
        #annotations=list(annotations.values())
        #annotations = annotations[0]
        for anno in annotations['annotations']:
          if anno['id'] is None:
            continue
          xmin = int(anno['x'])
          xmax = int(anno['x'] + anno['width'])
          ymin = int(anno['y'])
          ymax = int(anno['y'] + anno['height'])
          coors = [xmin, ymin, xmax, ymax]
          boxes.append(coors)
          classes.append(anno['id'])
                    
        width = 640
        height = 480
        
        return boxes, width, height, classes
        
    def load_mask(self, index):
        info = self.image_info[index]        
        anno_path = info['annotation']
        boxes,w,h, classes = self.extract_boxes(anno_path)
        masks = zeros([h,w,len(boxes)], dtype='uint8')
        class_ids = list()  
        for i in range(len(boxes)):
            obj = classes[i]
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index(obj))
        #info = self.image_info[1]
        return masks, asarray(class_ids, dtype='int32')
        
    def image_reference(self, index):
        info = self.image_info[index]        
        return info['path']
            


train_set = AridDataSet()    
train_set.load_dataset(is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))

test_set = AridDataSet()
test_set.load_dataset(is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))


#image_index = 0
#image = train_set.load_image(image_index)
#mask, class_ids = train_set.load_mask(image_index)
#pyplot.imshow(image)

#for j in range(mask.shape[2]):
#	pyplot.imshow(mask[:, :, j], cmap='gray', alpha=0.1)
## show the figure
#pyplot.show()
#bbox = extract_bboxes(mask)
# display image with masks and bounding boxes
#display_instances(image, bbox, mask, class_ids, train_set.class_names)



Train: 901
Test: 225


In [0]:
all_path = list()
for i in range(0, len(test_set.image_info)):
  all_path.append(test_set.image_info[i]["path"])

for i in range(0, len(train_set.image_info)):
  all_path.append(train_set.image_info[i]["path"])

import collections
#print([item for item, count in collections.Counter(all_path).items() if count > 1])
#print(all_path)

In [0]:
class maskRcnnConfig(Config):
    NAME = "MaskRCNN_config"
    
    # set the number of GPUs to use along with the number of images
    # per GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
 
    # number of classes (we would normally add +1 for the background)
     # kangaroo + BG
    NUM_CLASSES = 1+1
   
    # Number of training steps per epoch
    STEPS_PER_EPOCH = 131
    
    # Learning rate
    LEARNING_RATE=0.006
    
    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9
    
    # setting Max ground truth instances
    MAX_GT_INSTANCES=10
    
# define a configuration for the model
class AridConfig(Config):
  # Give the configuration a recognizable name
  NAME = "arid_cfg"
  # Number of classes (background + kangaroo)
  NUM_CLASSES = len(get_object_classes()) + 1
  # Number of training steps per epoch
  STEPS_PER_EPOCH = 160
  GPU_COUNT = 4
  IMAGES_PER_GPU = 1

    
# define the prediction configuration
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "arid_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = len(get_object_classes()) + 1
	# simplify GPU config
	GPU_COUNT = 4
	IMAGES_PER_GPU = 1

In [12]:
config = AridConfig()
config.display()
model = MaskRCNN(mode='training', model_dir='./Mask_RCNN', config=config)
#model.load_weights('/content/Mask_RCNN/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      4
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                74
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [0]:
class AridConfig_2(Config):
  # Give the configuration a recognizable name
  NAME = "arid_cfg"
  # Number of classes (background + kangaroo)
  NUM_CLASSES = len(get_object_classes()) + 1
  # Number of training steps per epoch
  STEPS_PER_EPOCH = 450
  GPU_COUNT = 4
  IMAGES_PER_GPU = 1

config = AridConfig_2()
config.display()
model = MaskRCNN(mode='training', model_dir='./Mask_RCNN', config=config)
model.load_weights('/content/Mask_RCNN/mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=40, layers='heads')


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     4
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      4
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                74
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:2033: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'




Epoch 1/40
449/450 [============================>.] - ETA: 1s - loss: 1.4575 - rpn_class_loss: 0.0306 - rpn_bbox_loss: 0.1361 - mrcnn_class_loss: 0.5670 - mrcnn_bbox_loss: 0.3086 - mrcnn_mask_loss: 0.4152WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:800: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



In [0]:
from mrcnn.model import load_image_gt
from numpy import mean

def evaluate_model(dataset, model, cfg):
	APs = list()
	precisionsList = list()
	class_ids = list()
	for image_id in dataset.image_ids:
		# load image, bounding boxes and masks for the image id
		image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset, cfg, image_id, use_mini_mask=False)
		class_ids.append(gt_class_id)
		# convert pixel values (e.g. center)
		scaled_image = mold_image(image, cfg)
		# convert image into one sample
		sample = expand_dims(scaled_image, 0)
		# make prediction
		yhat = model.detect(sample, verbose=0)
		# extract results for first sample
		r = yhat[0]
		# calculate statistics, including AP
		AP, precisions, _, _ = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])
		# store
		precisionsList.append(precisions)
		APs.append(AP)
	# calculate the mean AP across all images
	mAP = mean(APs)
	return mAP, precisionsList, class_ids

In [0]:
class PredictionConfig(Config):
	# define the name of the configuration
	NAME = "arid_cfg"
	# number of classes (background + kangaroo)
	NUM_CLASSES = len(get_object_classes()) + 1
	# simplify GPU config
	GPU_COUNT = 1
	IMAGES_PER_GPU = 1

cfg = PredictionConfig()
model = MaskRCNN(mode='inference', model_dir='./', config=cfg)
model.load_weights('/content/Mask_RCNN/Mask_RCNN/arid_cfg20200326T0927/mask_rcnn_arid_cfg_0040.h5', by_name=True)
# evaluate model on training dataset
train_mAP,_,_ = evaluate_model(train_set, model, cfg)
print("Train mAP: %.3f" % train_mAP)
# evaluate model on test dataset
test_mAP, test_precisionslist, class_ids = evaluate_model(test_set, model, cfg)
print("Test mAP: %.3f" % test_mAP)
print(test_precisionslist)

In [0]:
image_index = 0
classes = get_object_classes()
for image_id in test_set.image_ids:
  #_,_,_, classes = test_set.extract_boxes(anno_path)
  mapped_list = list(map(lambda x, y: str(x)+ ': ' +str(y), class_ids[image_id], test_precisionslist[image_id]))
  name = test_set.image_info[image_id]['path'].split('/')[-1].split('.')[0]
  #print(name, mapped_list)
for i in range(0, len(class_ids)):
  print(class_ids[i])
  print(test_precisionslist[i])
  print(test_set.image_info[i]['path'])

#print(len(test_set.image_info))
#print(list(map(lambda x, y: str(x)+ ': ' +str(y), range(1,len(classes)+2), classes)))